In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, GRU

gru_models = {}
gru_evals = {}
for crypto in splitted_data.keys():
    crypto_evals = {}
    crypto_models = {}
    for i in range(len(scenarios)):
        train_df = splitted_data[crypto]['trains'][i]
        test_df = splitted_data[crypto]['tests'][i]
        X_train, y_train = prepare_dl_data(train_df)
        X_test, y_test = prepare_dl_data(test_df)
        gru_model = Sequential()
        gru_model.add(GRU(64, input_shape=((X_train.shape[1], X_train.shape[2]))))
        gru_model.add(Dense(1))
        gru_model.compile(optimizer='adam', loss='mean_squared_error')
        early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
        gru_history = gru_model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test), callbacks=[early_stopping])
        predictions_gru = gru_model.predict(X_test)
        crypto_models[scenarios[i]] = gru_model
        crypto_evals[scenarios[i]] = eval(y_test, predictions_gru)
        forecast_plot('GRU', y_train, predictions, forecast_dl(crypto, 90, gru_model), crypto,scenarios[i], save=True)
    gru_evals[crypto] = crypto_evals
    gru_models[crypto] = crypto_models
    print(f'GRU | {crypto}')
display(pd.DataFrame(gru_evals))
pd.DataFrame(gru_evals).to_csv('GRU_evals.csv')